##Importing All necessary Packages

In [ ]:
# MAY NEED TO RESTART SESSION AFTER RUNNING THIS IF RUNNING ON GOOGLE COLAB
import os
os.chdir("/content/drive/MyDrive/AI_Projects/multimodal_enterprise_rag")
!pip install -e .

Obtaining file:///content/drive/MyDrive/AI_Projects/multimodal_enterprise_rag
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
INFO: pip is looking at multiple versions of langchain-community to determine which version is compatible with other requirements. This could take a while.
INFO: pip is looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
INFO: pip is still looking at multiple versions of langchain-google-genai to determine which version is compatible with other requirements. This could take a while.
INFO: This is taking longer than usual. You might need to provide the dependency resolver with stricter constraints to reduce runtime. See https://pip.pypa.io/warnings/backtracking for guidance. If you want to abort this run, press Ctrl + C.
INFO: pip is looking at multiple versi

In [ ]:
import os
import sys
os.chdir("/content/drive/MyDrive/AI_Projects/multimodal_enterprise_rag")
sys.path.append("/content/drive/MyDrive/AI_Projects/multimodal_enterprise_rag")

In [ ]:
!wget https://github.com/qdrant/qdrant/releases/download/v1.8.1/qdrant-x86_64-unknown-linux-gnu.tar.gz
!tar -xvf qdrant-x86_64-unknown-linux-gnu.tar.gz
!./qdrant/qdrant &

--2025-11-17 19:50:26--  https://github.com/qdrant/qdrant/releases/download/v1.8.1/qdrant-x86_64-unknown-linux-gnu.tar.gz
Resolving github.com (github.com)... 20.205.243.166
Connecting to github.com (github.com)|20.205.243.166|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://release-assets.githubusercontent.com/github-production-release-asset/268163609/50bec00a-f58a-4284-b8f1-689a0a22df6b?sp=r&sv=2018-11-09&sr=b&spr=https&se=2025-11-17T20%3A48%3A00Z&rscd=attachment%3B+filename%3Dqdrant-x86_64-unknown-linux-gnu.tar.gz&rsct=application%2Foctet-stream&skoid=96c2d410-5711-43a1-aedd-ab1947aa7ab0&sktid=398a6654-997b-47e9-b12b-9515b896b4de&skt=2025-11-17T19%3A47%3A58Z&ske=2025-11-17T20%3A48%3A00Z&sks=b&skv=2018-11-09&sig=6lCz5E4%2BMfrqGut2Kh8ITg9y5xobS%2Fm3CFSS%2F0rTXsA%3D&jwt=eyJ0eXAiOiJKV1QiLCJhbGciOiJIUzI1NiJ9.eyJpc3MiOiJnaXRodWIuY29tIiwiYXVkIjoicmVsZWFzZS1hc3NldHMuZ2l0aHVidXNlcmNvbnRlbnQuY29tIiwia2V5Ijoia2V5MSIsImV4cCI6MTc2MzQxMDgxOSwibmJmIjoxNzYzNDA5

#Ingestion Functions

##Pdf + text ingestion

In [ ]:
import pdfminer.high_level as pdf
from pathlib import Path

from langchain_text_splitters import RecursiveCharacterTextSplitter


from langchain_community.document_loaders import PDFMinerLoader
from langchain_core.documents import Document


import os
# def text_cleaning():
  #NEEDS TO BE DONE BEFORE TEXT SPLITTING (If time permits)
def text_split(path):
  print(os.getcwd())
  if path.lower().endswith(".pdf"):
    loader = PDFMinerLoader(path)
    docs = loader.load()
  elif path.lower().endswith(".txt"):
    with open(path, 'r') as f:
      pageContent = f.read()
    docs = Document(
      page_content=pageContent,
      metadata={
        "source_file": path,
        "modality": "text",
        "chunk_index": 0,
      }
    )

  if len(docs) == 0:
    raise ValueError("PDF text extraction failed or returned empty document.")
  #docs.page_content = text_cleaning(docs.page_content)

  text_splitter = RecursiveCharacterTextSplitter(
      chunk_size=1000,  # chunk size (characters)
      chunk_overlap=200,  # chunk overlap (characters) Starting 200 chars are the same as prev 200 chars to maintain context
      add_start_index=True,  # track index in original document
  )
  all_splits = text_splitter.split_documents(docs)


  print(f"Split blog post into {len(all_splits)} sub-documents.")
  return [Document(page_content=all_splits[i].page_content,metadata={**all_splits[i].metadata,"modality":"pdf","chunk_index":i}) for i in range(len(all_splits))]


In [ ]:
text_split("/content/drive/MyDrive/AI_Projects/multimodal_enterprise_rag/rag_system/ingestion/data/General_Resume.pdf")


/content/drive/MyDrive/AI_Projects/multimodal_enterprise_rag
Split blog post into 8 sub-documents.


[Document(metadata={'producer': 'Skia/PDF m141', 'creator': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/141.0.0.0 Safari/537.36', 'creationdate': '2025-11-08T19:35:27+00:00', 'title': 'General Resume - Google Docs', 'moddate': '2025-11-08T19:35:27+00:00', 'total_pages': 1, 'source': '/content/drive/MyDrive/AI_Projects/multimodal_enterprise_rag/rag_system/ingestion/data/General_Resume.pdf', 'start_index': 0, 'modality': 'pdf', 'chunk_index': 0}, page_content='Abhijeet Beedikar \n\n US Citizen | +16089102028  | beedikarabhijeet@gmail.com |  linkedin.com/in/abhijeetbeedikar/  |  github.com/AbhijeetBeedikar \n\n Education \n University of Wisconsin - Madison                                                       September 2024 -  May 2027 (Expected) \n\tB.S.\tTriple\tMajor\tin\tComputer\tScience,\tStatistics,\tand\tData\tScience\t|\tGPA:\t3.957\t|\tDean’s\tList\tFall\t2024,\tSpring\t2025\t\n Coursework:  Algorithms, Statistical Experimental Desig

##Image ingestion (OCR + Descriptive)

In [ ]:
import os
from PIL import Image
from io import BytesIO
import importlib
import rag_system.env as env
importlib.reload(env)

from langchain_google_genai import GoogleGenerativeAIEmbeddings
from langchain_core.documents import Document

from google.generativeai import configure, GenerativeModel

# Load .env variables (GOOGLE_API_KEY should be stored there)
os.environ["GOOGLE_API_KEY"] = env.key()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if GOOGLE_API_KEY is None:
    raise EnvironmentError("GOOGLE_API_KEY is not set. Add it to your .env file.")

configure(api_key=env.key())


# -----------------------------
# Utility: Load image safely
# -----------------------------
def load_image(path):
    try:
        return Image.open(path).convert("RGB")
    except Exception as e:
        raise ValueError(f"Failed to load image {path}: {e}")


# -----------------------------
# OCR using Gemini Vision
# -----------------------------
def run_ocr(image):
    """
    Uses Gemini 1.5 Flash OCR capability.
    Returns extracted text.
    """
    # REWORK: Use pytesseract library + Whisper for OCR
    model = GenerativeModel("gemini-2.5-flash")

    prompt = "Extract all readable text from this image. If no readable text exists, return an empty string. Do not add any extra text other than that which is asked for. Be concise in your answer."

    response = model.generate_content([prompt, image])
    if response.candidates[0].content.parts != []:
      return response.text.strip()
    else:
      return ""

# -----------------------------
# Captioning using Gemini Vision
# -----------------------------
def caption_image(image):
    """
    Generates a natural-language caption describing the image.
    """
    model = GenerativeModel("gemini-2.5-flash")

    prompt = (
        "Describe this image in detail in one sentence. "
        "Focus on what is visually present (objects, people, actions, scene)."
    )

    response = model.generate_content([prompt, image])

    return response.text.strip() if response.text else ""


# -----------------------------
# Object Detection (Gemini)
# -----------------------------
def detect_objects(image):
    """
    Uses Gemini to detect key objects.
    Gemini doesn't do bounding boxes but can classify objects visually.
    """
    model = GenerativeModel("gemini-2.5-flash")

    prompt = (
        "List all objects you see in this image. "
        "Return a comma-separated list of nouns only."
    )

    response = model.generate_content([prompt, image])

    if not response.text:
        return []

    # Convert "cat, skateboard, sunglasses" → ["cat", "skateboard", "sunglasses"]
    return [obj.strip().lower() for obj in response.text.split(",")]


# -----------------------------
# MAIN INGESTION FUNCTION
# -----------------------------
def ingest_image(path):
    """
    Ingest an image file:
    - Load image
    - OCR attempt
    - If OCR empty ➜ caption + object detection
    - Create a single text chunk
    - Return as langchain Document for embedding + vector DB indexing
    """

    image = load_image(path)
    file_name = os.path.basename(path)

    # --- Step 1: OCR ---
    ocr_text = run_ocr(image)

    if ocr_text:
        unified_text = ocr_text
        modality = "image-text"
        caption = None
        objects = []
        # REWORK: OCR Data should be chunked
    else:
        # --- Step 2: Captioning + Object Detection ---

        #REWORK: Use BLIP instead of Gemini
        caption = caption_image(image)
        objects = detect_objects(image)

        unified_text = (
            f"{caption}. Objects detected: {', '.join(objects)}"
            if objects
            else caption
        )

        modality = "image-visual"

    # --- Build Document ---
    doc = Document(
        page_content=unified_text,
        metadata={
            "source_file": file_name,
            "modality": modality,
            "caption": caption,
            "objects": objects,
            "ocr_text": ocr_text if ocr_text else None,
            "chunk_index": 0,
        }
    )

    return [doc]

In [ ]:
ingest_image("/content/drive/MyDrive/AI_Projects/multimodal_enterprise_rag/rag_system/ingestion/data/Descriptive.jpg")

ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 5377.95ms


[Document(metadata={'source_file': 'Descriptive.jpg', 'modality': 'image-visual', 'caption': 'Four distinct, powerful waterfalls cascade down steep, verdant cliffs amidst lush, mist-shrouded mountains, with distant buildings visible under an expansive, overcast grey sky.', 'objects': ['clouds', 'sky', 'waterfalls', 'water', 'mist', 'fog', 'mountains', 'hills', 'cliffs', 'rocks', 'trees', 'plants', 'bushes', 'leaves', 'branches', 'buildings', 'roofs', 'structures', 'earth'], 'ocr_text': None, 'chunk_index': 0}, page_content='Four distinct, powerful waterfalls cascade down steep, verdant cliffs amidst lush, mist-shrouded mountains, with distant buildings visible under an expansive, overcast grey sky.. Objects detected: clouds, sky, waterfalls, water, mist, fog, mountains, hills, cliffs, rocks, trees, plants, bushes, leaves, branches, buildings, roofs, structures, earth')]

##Audio Ingestion (Transcription)

In [ ]:
import os
import librosa
from langchain_core.documents import Document
from google.generativeai import GenerativeModel, configure
from google.genai.types import Blob
import importlib
import rag_system.ingestion.env as env
import pathlib
import mimetypes
importlib.reload(env)

# -------------------------
# Environment Setup
# -------------------------
os.environ["GOOGLE_API_KEY"] = env.key()
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
if GOOGLE_API_KEY is None:
    raise EnvironmentError("GOOGLE_API_KEY not found. Add it to your .env file.")

configure(api_key=GOOGLE_API_KEY)

def load_audio_blob(path):
    # Detect MIME type from the file extension
    mime_type = mimetypes.guess_type(path)[0]
    if mime_type is None:
        raise ValueError(f"Could not determine MIME type for file: {path}")

    # Load raw bytes from audio file
    with open(path, "rb") as f:
        audio_bytes = f.read()

    # Wrap in Gemini Blob format
    return {
        "mime_type": mime_type,
        "data": audio_bytes
    }
# -------------------------
# Load Audio File
# -------------------------
def load_audio(path):
    try:
        audio, sr = librosa.load(path, sr=None)
        return audio, sr
    except Exception as e:
        raise ValueError(f"Failed to load audio '{path}': {e}")


# -------------------------
# Transcribe using Gemini
# -------------------------
def transcribe_audio(path):
    """
    Returns transcription text.
    Gemini 1.5 Flash handles:
    - speech
    - noisy speech
    - accents
    - long-form speech
    """
    model = GenerativeModel("gemini-2.5-flash")

    prompt = (
        "Transcribe all spoken words from this audio. "
        "If the audio has no speech, return EXACTLY the string: 'NO_SPEECH'. Do not add any extra text other than that which is asked for. Be concise in your answer."
    )
    audio_bytes = pathlib.Path(path).read_bytes()
    audio_blob = Blob(
        data=audio_bytes,
        mime_type='audio/mp3'
    )
    response = model.generate_content([prompt, load_audio_blob(path)])

    return response.text.strip() if response.text else "NO_SPEECH"


# -------------------------
# Caption Non-Speech Audio
# -------------------------
def caption_audio(path):
    """
    Describe what you hear if there is no speech.
    Good for:
    - music
    - ambient noise
    - environmental soundscapes
    - mechanical noises
    """
    model = GenerativeModel("gemini-2.5-flash")
    prompt = (
        "Describe this audio clip in one detailed sentence. "
        "Do not invent speech if none exists."
    )

    with open(path, "rb") as f:
        audio_bytes = f.read()

    response = model.generate_content([prompt, audio_bytes])
    return response.text.strip() if response.text else ""


# -------------------------
# Chunk Text if Long
# -------------------------
def chunk_text(text, max_len=600, overlap=100):
    """
    Simple text chunker for audio transcripts.
    """
    words = text.split()
    chunks = []

    start = 0
    while start < len(words):
        end = start + max_len
        chunk_words = words[start:end]
        chunk_text = " ".join(chunk_words)
        chunks.append(chunk_text)
        start += max_len - overlap

    return chunks


# -------------------------
# MAIN INGEST FUNCTION
# -------------------------
def ingest_audio(path):
    """
    Ingest audio by:
    - transcribing speech
    - falling back to captioning for music/noise
    - chunking long transcripts
    - returning LangChain Document objects
    """

    file_name = os.path.basename(path)

    # Load audio to get duration
    audio, sr = load_audio(path)
    duration_seconds = librosa.get_duration(y=audio, sr=sr)

    # Step 1: Try transcription
    transcription = transcribe_audio(path)

    if transcription != "NO_SPEECH" and transcription.strip():
        modality = "audio-speech"
        base_text = transcription
    else:
        # Step 2: Fallback to captioning
        caption = caption_audio(path)
        base_text = caption
        modality = "audio-non-speech"

    # Step 3: Chunking (only if long)
    chunks = chunk_text(base_text) if len(base_text) > 700 else [base_text]

    # Step 4: Build Document objects
    docs = []
    for idx, chunk in enumerate(chunks):
        doc = Document(
            page_content=chunk,
            metadata={
                "source_file": file_name,
                "modality": modality,
                "duration_seconds": duration_seconds,
                "chunk_index": idx,
                "num_chunks": len(chunks)
            }
        )
        docs.append(doc)

    return docs


In [ ]:
ingest_audio("/content/drive/MyDrive/AI_Projects/multimodal_enterprise_rag/rag_system/ingestion/data/WordsAudio.mp3")

[Document(metadata={'source_file': 'WordsAudio.mp3', 'modality': 'audio-speech', 'duration_seconds': 12.095986394557823, 'chunk_index': 0, 'num_chunks': 1}, page_content='And uh customs like new clothes and sharing sweets ground us in our culture even though the degree of celebration differs. Uh it reminds us about our country even from a distance.')]

#Indexing Vector Database

In [ ]:
import os
from langchain.chat_models import init_chat_model
from langchain_google_genai import GoogleGenerativeAIEmbeddings

from qdrant_client.models import Distance, VectorParams
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
import getpass
import importlib
import rag_system.env as env
importlib.reload(env)

from qdrant_client import QdrantClient
from qdrant_client.models import VectorParams, Distance, PointStruct
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
import shutil


# ---------------------------------------
# Initialize Qdrant Collection
# ---------------------------------------
def init_qdrant(collection_name="multimodal_rag", path="/content/drive/MyDrive/AI_Projects/multimodal_enterprise_rag/qdrant_data"):
    """
    Use a persistent local Qdrant instance, saved inside your repo folder.
    """
    shutil.rmtree(path, ignore_errors=True) # remove existing Database (because only 1 database can be running at a time in the same directory)
    client = QdrantClient(path=path) #Qdrant Vector database is stored at path

    embedding_model = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

    vector_size = len(embedding_model.embed_query("hello world")) # to get size of embedding returned by the model for some text

    if not client.collection_exists(collection_name): # if the collection doesnt already exist
        client.create_collection(
            collection_name=collection_name, # name of the collection
            vectors_config=VectorParams(
                size=vector_size, # Size of all points within collection = vector_size
                distance=Distance.COSINE # comparison parameter for all points = cosine similarity
            )
        )

    return client


# ---------------------------------------
# Convert LangChain Document → Qdrant Point
# ---------------------------------------
def document_to_point(doc, idx, embedding_model):
    """
    Converts a Document into:
      - vector embedding
      - metadata dict
      - unique ID
    """
    embedding = embedding_model.embed_query(doc.page_content) #creating the embedding itself for each text chunk doc
    payload = doc.metadata.copy()
    payload["text"] = doc.page_content  # store raw text for extraction

    return PointStruct(
        id=idx, # id of each
        vector=embedding,
        payload=payload
    )


# ---------------------------------------
# Main Index Function
# ---------------------------------------
def index_documents(docs, collection_name="multimodal_rag", persist_path="/content/drive/MyDrive/AI_Projects/multimodal_enterprise_rag/qdrant_data"):
    """
    docs: list[Document]
    """
    if len(docs) == 0:
        raise ValueError("No documents provided for indexing")

    # 1. Initialize Qdrant
    client = init_qdrant(collection_name=collection_name, path=persist_path)

    # 2. Create embeddings
    embedding_model = GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001")

    # 3. Convert docs → Qdrant points
    points = []
    for idx, doc in enumerate(docs):
        point = document_to_point(doc, idx, embedding_model)
        points.append(point)

    # 4. Upsert into Qdrant (vector + payload)
    client.upsert(
        collection_name=collection_name,
        points=points
    )

    print(f"Indexed {len(docs)} documents into Qdrant collection '{collection_name}'.")

    return (client,collection_name)


In [ ]:
a = ingest_image("/content/drive/MyDrive/AI_Projects/multimodal_enterprise_rag/rag_system/ingestion/data/Descriptive.jpg")
index_documents(a)

Indexed 1 documents into Qdrant collection 'multimodal_rag'.


(<qdrant_client.qdrant_client.QdrantClient at 0x79b2240668d0>,
 'multimodal_rag')

#Rag-pipeline for abstractly handling ingesting and indexing

In [ ]:
import rag_system.ingestion.pdfingestion as pdfingestion
import rag_system.ingestion.img_ingest as img_ingest
import rag_system.ingestion.audio_ingest as audio_ingest
import importlib
importlib.reload(pdfingestion)
importlib.reload(img_ingest)
importlib.reload(audio_ingest)
#from rag_system.ingestion.ingest_video import ingest_video

import rag_system.vector_db.indexing as indexing
importlib.reload(indexing)

#from rag_system.retrieval.hybrid_search import hybrid_search


def ingest_and_index(path):
    if path.endswith(".pdf"):
        docs = pdfingestion.text_split(path)
    elif path.lower().endswith(('.png', '.jpg', '.jpeg')):
        docs = img_ingest.ingest_image(path)
    elif path.lower().endswith(('.mp3', '.wav', '.m4a')):
        docs = audio_ingest.ingest_audio(path)
    #elif path.lower().endswith(('.mp4', '.mov', '.avi')):
     #   docs = ingest_video(path)
    else:
        raise ValueError("Unsupported file format")
    print("Ingestion + Indexing complete.")
    return indexing.index_documents(docs)

In [ ]:
ingest_and_index("/content/drive/MyDrive/AI_Projects/multimodal_enterprise_rag/rag_system/ingestion/data/General_Resume.pdf")


/content/drive/MyDrive/AI_Projects/multimodal_enterprise_rag/rag_system/graph
Split blog post into 8 sub-documents.
Ingestion + Indexing complete.
Indexed 8 documents into Qdrant collection 'multimodal_rag'.


(<qdrant_client.qdrant_client.QdrantClient at 0x79b2029ce000>,
 'multimodal_rag')

# Graph Functions

## Graph Building

In [ ]:
import google.generativeai as genai
import json
import importlib
import rag_system.env as env
importlib.reload(env)
import os
import networkx as nx
import pickle
import pathlib
G = nx.DiGraph()   # directed graph
os.environ["GOOGLE_API_KEY"] = env.key()
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])
model = genai.GenerativeModel("gemini-2.5-flash")


def extract_entities_and_relations(text):
    prompt = f'''
    Extract entities and relationships from the following text.Write the response as a plain text output. that only consists of the JSON contents. Strictly nothing more. The response must start and end with a curly bracket.

    Return JSON in this exact format:
    {{
      "entities": ["ENTITY1", "ENTITY2", ...],
      "relationships": [
        {{"source": "ENTITY1", "target": "ENTITY2", "relation": "RELATIONSHIP"}}
      ]
    }}

    TEXT:
    {text}
    '''

    res = model.generate_content(prompt)
    load_text = res.text[res.text.index("{"):][::-1]
    load_text = load_text[load_text.index("}"):][::-1]
    return json.loads(load_text)


def add_to_graph(entity_data):
    entities = entity_data["entities"]
    relationships = entity_data["relationships"]

    # Add nodes
    for e in entities:
        G.add_node(e, type="entity")

    # Add edges
    for rel in relationships:
        src = rel["source"]
        trg = rel["target"]
        r   = rel["relation"]

        G.add_edge(src, trg, relation=r)
def process_document_for_graph(text):
    extracted = extract_entities_and_relations(text)
    add_to_graph(extracted)
    os.chdir("/content/drive/MyDrive/AI_Projects/multimodal_enterprise_rag/rag_system/graph")
    with open("knowledge_graph.gpickle", "wb") as f:
      pickle.dump(G, f, pickle.HIGHEST_PROTOCOL)
    if not pathlib.Path("graph_functions.py").exists():
      with open("graph_functions.py", "w") as f:
        f.write('''import pickle
  import os
  os.chdir("/content/drive/MyDrive/AI_Projects/multimodal_enterprise_rag/rag_system/graph")
  with open("knowledge_graph.gpickle", "rb") as f:
    G = pickle.load(f)

  def list_all_entities():
    return list(G.nodes())

  def list_all_relations(entity=None):
    if entity is None:
      relations = []
      for u, v, data in G.edges(data=True):
        relations.append(f"{u} —[{data['relation']}]-> {v}")
      return relations
    else:
      return list(G.out_edges(entity, data=True))

  def neighbours(entity):
    return list(G.neighbors(entity))

  for u, v, data in G.edges(data=True):
      print(f"{u} —[{data['relation']}]-> {v}")

  def get_graph():
    return G''')
    return extracted


In [ ]:
text = """
John Smith is the founder of Acme Corp. The company operates in San Francisco.
"""
process_document_for_graph(text)

{'entities': ['John Smith', 'Acme Corp', 'San Francisco'],
 'relationships': [{'source': 'John Smith',
   'target': 'Acme Corp',
   'relation': 'founder of'},
  {'source': 'Acme Corp',
   'target': 'San Francisco',
   'relation': 'operates in'}]}

##Graph Searching

In [ ]:
"""
Graph Search Module
--------------------
Provides:
1. Exact entity search
2. Relationship traversal (1-hop, multi-hop)
3. Keyword search inside graph nodes and edges
"""

import networkx as nx
from typing import List, Dict, Any, Set


# ------------------------------------------------------------
# EXACT ENTITY SEARCH
# ------------------------------------------------------------
def search_entity(G: nx.DiGraph, entity_name: str):
    """
    Return the exact entity node if it exists in the graph.
    """
    entity_name = entity_name.strip()
    if entity_name in G.nodes:
        return entity_name
    return None


# ------------------------------------------------------------
# RELATIONSHIP TRAVERSAL
# ------------------------------------------------------------
def one_hop_relations(G: nx.DiGraph, entity: str) -> List[Dict[str, str]]:
    """
    Returns all relationships where the entity is the source or target.
    Example output:
    [
      {"source": "A", "relation": "works_at", "target": "Company"}
    ]
    """
    relations = []

    # Outbound edges (entity -> other)
    for _, target, data in G.out_edges(entity, data=True):
        relations.append({
            "source": entity,
            "relation": data.get("relation", ""),
            "target": target
        })

    # Inbound edges (other -> entity)
    for source, _, data in G.in_edges(entity, data=True):
        relations.append({
            "source": source,
            "relation": data.get("relation", ""),
            "target": entity
        })

    return relations


def multi_hop_traversal(G: nx.DiGraph, start_entity: str, depth: int = 2) -> Set[str]:
    """
    BFS traversal up to 'depth' hops.
    Returns all entities reachable within k hops.
    """
    visited = set([start_entity])
    frontier = set([start_entity])

    for _ in range(depth):
        next_frontier = set()
        for node in frontier:
            # neighbors = outbound (G.neighbors returns outbound nodes by default) U inbound nodes (predecessors)
            neighbors = set(G.neighbors(node)) | set(G.predecessors(node))
            next_frontier |= neighbors

        next_frontier -= visited # already visited nodes must not be added to visited set
        visited |= next_frontier # all unique nodes in the _th depth are added to the visited set
        frontier = next_frontier # becomes the set of next nodes to come from which we need to traverse through 1 more BFS iteration

    visited.remove(start_entity)
    return visited


# ------------------------------------------------------------
# KEYWORD SEARCH INSIDE GRAPH
# ------------------------------------------------------------
def keyword_search(G: nx.DiGraph, keyword: str) -> Dict[str, Any]:
    """
    Search for keyword in:
    - node names
    - node attributes
    - edge relation labels
    Returns all matched nodes + edges.
    """
    keyword = keyword.lower()
    matched_nodes = []
    matched_edges = []

    # Node search
    for node, attrs in G.nodes(data=True):
        if keyword in node.lower():
            matched_nodes.append(node)
            continue

        # Search inside node attributes
        for k, v in attrs.items():
            if isinstance(v, str) and keyword in v.lower():
                matched_nodes.append(node)
                break

    # Edge search
    for u, v, attrs in G.edges(data=True):
        relation = attrs.get("relation", "")
        if keyword in relation.lower():
            matched_edges.append({
                "source": u,
                "relation": relation,
                "target": v
            })

    return {
        "nodes": list(set(matched_nodes)), #set of nodes that contain the keyword
        "edges": matched_edges #set of edges that contain the keyword
    }


# ------------------------------------------------------------
# COMBINED GRAPH QUERY (optional helper)
# ------------------------------------------------------------
def graph_query(G: nx.DiGraph, query: str) -> Dict[str, Any]:
    """
    Helper function that:
    1. Tries exact entity search
    2. If not found → keyword search
    3. Returns local graph neighborhood if entity is found
    """
    result = {}

    # Exact entity match
    entity = search_entity(G, query)
    if entity:
        result["entity"] = entity
        result["one_hop"] = one_hop_relations(G, entity)
        result["multi_hop"] = list(multi_hop_traversal(G, entity, depth=2))
        return result

    # If no exact match → keyword search
    result["keyword_matches"] = keyword_search(G, query)
    return result


In [ ]:
import os
import pickle
text = """
John Smith is the founder of Acme Corp. The company operates in San Francisco.
"""
print(process_document_for_graph(text))
os.chdir("/content/drive/MyDrive/AI_Projects/multimodal_enterprise_rag/rag_system/graph")
with open("knowledge_graph.gpickle", "rb") as f:
  G = pickle.load(f)
graph_query(G,"Acme Corp")


ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 863.16ms


{'entities': ['John Smith', 'Acme Corp', 'San Francisco'], 'relationships': [{'source': 'John Smith', 'target': 'Acme Corp', 'relation': 'founder of'}, {'source': 'Acme Corp', 'target': 'San Francisco', 'relation': 'operates in'}]}


{'entity': 'Acme Corp',
 'one_hop': [{'source': 'Acme Corp',
   'relation': 'operates in',
   'target': 'San Francisco'},
  {'source': 'John Smith', 'relation': 'founder of', 'target': 'Acme Corp'}],
 'multi_hop': ['San Francisco', 'John Smith']}

#Orchestrator to run Hybrid Search

In [ ]:
"""
HYBRID SEARCH ORCHESTRATOR
--------------------------
Combines:
  • Query classification
  • Graph search (exact + multi-hop)
  • Vector search (Qdrant)
  • Keyword expansion
  • LLM answer generation

This file uses FUNCTIONS ONLY (no classes).
"""

import google.generativeai as genai

from sentence_transformers import SentenceTransformer, util
from langchain_google_genai.embeddings import GoogleGenerativeAIEmbeddings
import pickle
import importlib
import rag_system.env as env
importlib.reload(env)
import os
os.environ["GOOGLE_API_KEY"] = env.key()
genai.configure(api_key=os.environ["GOOGLE_API_KEY"])


# -------------------------
# COSINE SIMILARITY CLASSIFIER
# -------------------------
def classify_query(query: str):
    model = SentenceTransformer("all-MiniLM-L6-v2")   # fast, light

    # -------------------------
    # CATEGORY DESCRIPTIONS
    # -------------------------
    CATEGORIES = {
        "FACTUAL_LOOKUP": """
            Direct factual questions requiring a specific answer.
            Examples: who, when, where, what, founder, CEO, location.
        """,

        "SUMMARIZATION": """
            Requests to summarize, explain, outline, or describe text broadly.
            Examples: summarize, overview, high level, explain.
        """,

        "RELATIONAL_REASONING": """
            Questions about relationships or comparisons between entities.
            Examples: relationship between X and Y, compare, how connected.
        """,

        "CROSS_MODAL_LINKAGE": """
            Queries linking text, image, audio, or video content.
            Examples: describe the image, relate audio to document, show connected context.
        """,

        "KEYWORD_SEARCH": """
            Pure keyword-based search or filtering.
            Examples: find mentions of X, occurrences of Y, keyword search.
        """
    }

    # converts each category description to tensors to represent them numerically
    category_embeddings = {
      name: model.encode(desc, convert_to_tensor=True) for name, desc in CATEGORIES.items()
    }

    #converts the query to tensors
    query_emb = model.encode(query, convert_to_tensor=True)

    best_cat = None
    best_score = -1
    #REWORK: What if there are multiple categories mentioned? Keep a threshold beyond which catgeories are accepted
    for cat_name, cat_emb in category_embeddings.items():

        # get cosine similarity score for each category
        score = util.cos_sim(query_emb, cat_emb).item()
        if score > best_score:
            best_score = score
            best_cat = cat_name

    return best_cat, best_score

# ----------------------------------------
# 2. Vector Search
# ----------------------------------------

def vector_search(query, client, embedder= GoogleGenerativeAIEmbeddings(model="models/gemini-embedding-001"), collection="multimodal_rag", k=5):
    qvec = embedder.embed_query(query)

    results = client.query_points(
        collection_name=collection,
        query=qvec,
        limit=k,
        with_payload=True
    )

    return [hit.payload["text"] for hit in results.points]



# ----------------------------------------
# 2. Hybrid Graph + Vector Retrieval
# ----------------------------------------
def hybrid_retrieve(query: str, client, G):

    """
    Performs:
      • entity extraction from query
      • graph search
      • vector DB retrieval
      • merging of results
    """

    # --- Extract entities from text ---
    ner = extract_entities_and_relations(query)
    entities = ner.get("entities", []) # [] is the default list returned

    graph_results = []
    related_entities = set()

    # ---- 1. Try exact graph match ----
    for e in entities:
        exact = search_entity(G, e)
        if exact:
            # 1-hop relations
            hops1 = one_hop_relations(G, exact)

            # 2-hop neighbors
            hops2 = multi_hop_traversal(G, exact, depth=2)

            graph_results.append({
                "entity": exact,
                "one_hop": hops1,
                "two_hop": list(hops2)
            })

            # gather all nodes discovered
            related_entities.update([h["target"] for h in hops1] + [hs["source"] for hs in hops1])
            related_entities.update(hops2)

    # ---- 2. Keyword search fallback to check for the existence of a word within nodes/edges in the Network graph ----

    if not graph_results:
        for token in query.split():
            ks = keyword_search(G, token)
            if ks["nodes"] or ks["edges"]:
                graph_results.append({"keyword_matches": ks})
                related_entities.update(ks["nodes"])
                if ks["edges"]:
                    related_entities.update([t["target"] for t in ks["edges"]] + [s["source"] for s in ks["edges"]])


    # ---- 3. Vector DB search ----
    vector_hits = vector_search(
        query=query,
        client=client,
    )

    # also expand vector search using related graph entities
    for ent in related_entities:
        extra_hits = vector_search(
            query=ent,
            client=client,
            k=2
        )
        vector_hits.extend(extra_hits)

    # remove duplicates from vector_hits
    #seen = set()
    final_context = list(set(vector_hits))
   # for hit in vector_hits:
    #    text = hit
     #   if text not in seen:
      #      seen.add(text)
       #     final_context.append(text)

    # Final merged result
    return {
        "graph": graph_results, # nodes related to the query within Knowledge Graph
        "contexts": final_context # Qdrant Point texts similar to the query and Qdrant Point texts similar to other texts related to the query (extracted from nodes neighboring the query entity in Knowledge Graph)
    }

# ----------------------------------------
# 3. Main RAG Orchestration Function
# ----------------------------------------
def orchestrate(query: str, client, G):
    """
    Full hybrid retrieval + LLM answer.
    """

    query_type = classify_query(query)

    # --- Summarization → Only vector search ---
    if query_type == "summarization":
        vec_only = vector_search(query=query, client=client, k=8)
        context = "\n".join(hit.payload["text"] for hit in vec_only)
        prompt = f"""
    You are a helpful Assistant. Answer the user query using the context below.

    CONTEXT:
    {context}

    QUERY:
    {query}

    Be accurate and concise. Do not hallucinate.
    """
        response = genai.GenerativeModel("gemini-2.5-flash").generate_content(prompt).text
        return {"response":response,"context":[hit.payload["text"] for hit in vec_only]}

    # --- Lookup or cross-modal → Use hybrid graph + vector ---
    hybrid = hybrid_retrieve(query, client, G)
    context = "\n".join(hybrid["contexts"])

    # Combine graph info with context
    graph_context = str(hybrid["graph"])

    final_prompt = f"""
    You are a helpful Assistant. Answer the user query using the context below.

    CONTEXT:
    {context}

    GRAPH CONTEXT:
    {graph_context}

    QUERY:
    {query}

    Be accurate and concise. Do not hallucinate.
    """
    response = genai.GenerativeModel("gemini-2.5-flash").generate_content(final_prompt).text
    return {"response":response,"context":hybrid["contexts"]}


In [ ]:
import pickle
import os
import shutil
os.chdir("/content/drive/MyDrive/AI_Projects/multimodal_enterprise_rag/rag_system/graph")
with open("knowledge_graph.gpickle", "rb") as f:
  G = pickle.load(f)
shutil.rmtree("/content/drive/MyDrive/AI_Projects/multimodal_enterprise_rag/qdrant_data", ignore_errors=True) # remove existing Database (because only 1 database can be running at a time in the same directory)
print("image_ingestion started")
a = ingest_image("/content/drive/MyDrive/AI_Projects/multimodal_enterprise_rag/rag_system/ingestion/data/Descriptive.jpg")
print("image_ingestion done")
client = index_documents(a)[0]
print(client.get_collections())
orchestrate("who is the founder of Acme Corp", client, G)["response"]

image_ingestion started


ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 8031.28ms
ERROR:tornado.access:503 POST /v1beta/models/gemini-2.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint (::1) 6244.35ms


image_ingestion done
Indexed 1 documents into Qdrant collection 'multimodal_rag'.
collections=[CollectionDescription(name='multimodal_rag')]


'John Smith'

#Main Function

In [ ]:
import os
import pickle
def get_all_text_from_collection(client, collection_name):
    collected = []
    scroll_filter = None
    next_page = None

    while True:
        points, next_page = client.scroll(
            collection_name=collection_name,
            limit=200,    # Qdrant default max
            offset=next_page,
            with_payload=True,
            with_vectors=False
        )

        if not points:
            break

        for p in points:
            # assuming you stored text under payload["text"]
            if "text" in p.payload:
                collected.append(p.payload["text"])

        if next_page is None:
            break

    return collected
def user_input(mediapath,query):
  # Filling the Vector Database
  (client,collection_name) = ingest_and_index(mediapath)
  print("ingested and indexed")

  # Building the Graph
  corpus_text = get_all_text_from_collection(client, collection_name)
  print("corpus text obtained")
  for i in corpus_text:
    process_document_for_graph(i)

  # loading updated graph for hybrid retrieval
  os.chdir("/content/drive/MyDrive/AI_Projects/multimodal_enterprise_rag/rag_system/graph")
  with open("knowledge_graph.gpickle", "rb") as f:
    G = pickle.load(f)
  return orchestrate(query, client, G) # returns response


In [ ]:
user_input("/content/drive/MyDrive/AI_Projects/multimodal_enterprise_rag/rag_system/ingestion/data/OCR.png","What is the likelihood of th echip being faulty?")["response"]

Ingestion + Indexing complete.
Indexed 1 documents into Qdrant collection 'multimodal_rag'.
ingested and indexed
corpus text obtained


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

'If the outcome of a particular test indicates that there is a defect, the likelihood that the chip is actually faulty is approximately **0.047**.\n\nThe inherent probability of a chip having a defect (without any test information) is **0.001** (1 out of every 1000 chips).'

#DeepEval LLM Evaluation

In [ ]:
from deepeval.test_case import LLMTestCase
from deepeval.metrics import FaithfulnessMetric, GEval, HallucinationMetric
from deepeval import evaluate

def test_rag_pipeline():
    # Test example ------------------------
    query = "What ground us in our culture?"
    media_path = "/content/drive/MyDrive/AI_Projects/multimodal_enterprise_rag/rag_system/ingestion/data/WordsAudio.mp3"

    # Run your full pipeline
    result = user_input(media_path, query)


    # You must manually provide ground truth
    ground_truth = "The customs, like new clothes, and sharing sweets."


    # DeepEval test case -------------------
    test_case = LLMTestCase(
        input = query,
        expected_output = ground_truth,
        generated_answer = result["response"],
        context = result["context"],
    )

    metrics = [
      FaithfulnessMetric(),
      HallucinationMetric(),
      GEval(),
    ]

    eval = []
    for i in range(len(metrics)):
     eval.append(metrics[i].score(test_case))

    return eval


In [ ]:
import importlib
import rag_system.main as main
importlib.reload(main)
main.user_input("/content/drive/MyDrive/AI_Projects/multimodal_enterprise_rag/rag_system/ingestion/data/WordsAudio.mp3","What ground us in our culture?")


Ingestion + Indexing complete.
Indexed 1 documents into Qdrant collection 'multimodal_rag'.
ingested and indexed
corpus text obtained


{'response': 'Customs like new clothes and sharing sweets ground us in our culture.',
 'context': ['And customs like new clothes and sharing sweets ground us in our culture even though the degree of celebration differs. It reminds us about our country even from a distance.']}